In [1]:
import cv2
import matplotlib as plt
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

img_path = '../data/kkj/kkj04_lipsync3d/crop/00003.png'
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:

    image = cv2.imread(img_path)
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    for idx, landmark in enumerate(results.multi_face_landmarks[0].landmark):
          print('idx: {}, landmark: {}'.format(idx, landmark))

    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          landmark_drawing_spec=drawing_spec,
          connections=None,
          connection_drawing_spec=None)
        
        nose = face_landmarks.landmark[2]
        left_eye = face_landmarks.landmark[130]
        right_eye = face_landmarks.landmark[359]

        image_rows, image_cols, _ = image.shape
        nose = mp_drawing._normalized_to_pixel_coordinates(nose.x, nose.y, image_cols, image_rows)
        left_eye = mp_drawing._normalized_to_pixel_coordinates(left_eye.x, left_eye.y, image_cols, image_rows)
        right_eye = mp_drawing._normalized_to_pixel_coordinates(right_eye.x, right_eye.y, image_cols, image_rows)

        cv2.circle(annotated_image, nose, 2, mp_drawing.RED_COLOR, 2)
        cv2.circle(annotated_image, left_eye, 2, mp_drawing.RED_COLOR, 2)
        cv2.circle(annotated_image, right_eye, 2, mp_drawing.RED_COLOR, 2)

    cv2.imwrite('annotated_image.png', annotated_image)

idx: 0, landmark: x: 0.5957899689674377
y: 0.44643381237983704
z: -0.050491347908973694

idx: 1, landmark: x: 0.5920318961143494
y: 0.3926335871219635
z: -0.07664436101913452

idx: 2, landmark: x: 0.5941587686538696
y: 0.4120352864265442
z: -0.045612744987010956

idx: 3, landmark: x: 0.5788167715072632
y: 0.34754398465156555
z: -0.05007738620042801

idx: 4, landmark: x: 0.5909350514411926
y: 0.3768664002418518
z: -0.07927119731903076

idx: 5, landmark: x: 0.5903090834617615
y: 0.3585273325443268
z: -0.0709119588136673

idx: 6, landmark: x: 0.5894942283630371
y: 0.31734180450439453
z: -0.025578901171684265

idx: 7, landmark: x: 0.49803847074508667
y: 0.32494160532951355
z: 0.034920018166303635

idx: 8, landmark: x: 0.588245689868927
y: 0.2887633442878723
z: -0.010640733875334263

idx: 9, landmark: x: 0.5872071385383606
y: 0.27063417434692383
z: -0.009299964644014835

idx: 10, landmark: x: 0.5845979452133179
y: 0.2065565288066864
z: 0.019861901178956032

idx: 11, landmark: x: 0.596339046

In [101]:
import numpy as np

tgt_points = np.random.rand(10, 3)
gt_R = np.random.rand(3, 3)
gt_t = np.random.rand(3, 1)
gt_c = 1.0
ref_points = (gt_c * np.matmul(gt_R, tgt_points.T) + gt_t).T


ref_mu = ref_points.mean(axis=0)
tgt_mu = tgt_points.mean(axis=0)
ref_var = ref_points.var(axis=0).sum()
tgt_var = tgt_points.var(axis=0).sum()
n = ref_points.shape[0]
covar = np.matmul((ref_points - ref_mu).T, tgt_points - tgt_mu) / n

det_covar = np.linalg.det(covar)
u, d, vh = np.linalg.svd(covar)
detuv = np.linalg.det(u) * np.linalg.det(vh.T)


if detuv>= 0:
    S = np.identity(3)
else:
    S = np.diag([1, 1, -1])
R = np.matmul(np.matmul(u, S), vh)
c = (1 / tgt_var) * np.trace(np.matmul(np.diag(d), S))
t = ref_mu.reshape(3, 1) - c * np.matmul(R, tgt_mu.reshape(3, 1))
print(gt_R)
print(R)
print(c)
print(t)

[[0.03708155 0.04868199 0.34324353]
 [0.86332402 0.85508935 0.23015157]
 [0.15138852 0.51759198 0.50496079]]
[[-0.06585066 -0.60573752  0.7929349 ]
 [ 0.98510032  0.08706061  0.14831657]
 [-0.15887431  0.79088717  0.59097922]]
0.5084289343592447
[[0.54166481]
 [1.3276572 ]
 [1.22011957]]


In [102]:
print(np.sum(ref_points - (gt_c * np.matmul(gt_R, tgt_points.T) + gt_t).T))
print(np.sum(ref_points - (c * np.matmul(R, tgt_points.T) + t).T))
print(ref_points)
print('--------------------------------')
print((gt_c * np.matmul(gt_R, tgt_points.T) + gt_t).T)
print('--------------------------------')
print((c * np.matmul(R, tgt_points.T) + t).T)

0.0
0.0
[[0.53466867 2.06277533 1.50961432]
 [0.53463201 1.60240775 1.301713  ]
 [0.51606737 1.52204722 1.44110627]
 [0.61869544 1.62370932 1.52926292]
 [0.71037131 1.9087528  1.63683988]
 [0.58955624 1.81453409 1.41499704]
 [0.55540605 1.63832042 1.59645968]
 [0.44179716 1.16194283 1.20274253]
 [0.47018801 1.29553475 1.20586545]
 [0.5150573  1.68358688 1.46519269]]
--------------------------------
[[0.53466867 2.06277533 1.50961432]
 [0.53463201 1.60240775 1.301713  ]
 [0.51606737 1.52204722 1.44110627]
 [0.61869544 1.62370932 1.52926292]
 [0.71037131 1.9087528  1.63683988]
 [0.58955624 1.81453409 1.41499704]
 [0.55540605 1.63832042 1.59645968]
 [0.44179716 1.16194283 1.20274253]
 [0.47018801 1.29553475 1.20586545]
 [0.5150573  1.68358688 1.46519269]]
--------------------------------
[[0.4513967  1.82467044 1.42415902]
 [0.62127127 1.7523462  1.27891677]
 [0.46169088 1.48470742 1.50585408]
 [0.63417773 1.61191569 1.50194258]
 [0.76513507 1.83148964 1.48079201]
 [0.6595797  1.85174461 

In [33]:
from utils import Umeyama_algorithm

ref = {2: [0, 0, 0], 130: [5, 0, 0], 359: [0, 5, 0]}
tgt = {2: [0, 0, 7], 130: [5, 0, 7], 359: [0, 5, 6]}

R, t, c = Umeyama_algorithm(ref,tgt)

ref:  [[0 0 0]
 [5 0 0]
 [0 5 0]]
tgt:  [[0 0 7]
 [5 0 7]
 [0 5 6]]


In [34]:
import numpy as np
tgt_points = np.array([tgt[2], tgt[130], tgt[359]])
(np.matmul(R, tgt_points.T) + np.matmul(t, np.ones((1, 3)))).T

array([[ 2.48166669e-02, -3.78439056e-02,  6.78729003e-02],
       [ 5.02475658e+00, -1.33320399e-02,  6.78729003e-02],
       [-1.80629353e-04,  5.06111433e+00,  6.78729003e-02]])

In [37]:
R

array([[ 9.99987983e-01, -4.80717235e-03,  9.61434470e-04],
       [ 4.90237313e-03,  9.80568892e-01, -1.96113778e-01],
       [ 0.00000000e+00,  1.96116135e-01,  9.80580676e-01]])